# Transportation and Analytics Homework 1
## By Austin Slakey
<br>

### PROBLEM 1
One way to think about a graph G is with a square matrix C. The entry cuv denotes the cost of an edge from u to v. Consider the graph in the matrix in C.csv.<br><br>
a) Write code that implement’s Dijkstra’s Algorithm. What is the shortest path from 1 to 20, and how much does it cost? What is the shortest path from 11 to 13, and how much does it cost? What is the shortest path from 17 to 3, and how much does it cost?<br><br>
b) Repeat part a), but now you can only use at most 3 steps. What if I can use 5 steps?

In [4]:
import pandas as pd

#read CSV file
df = pd.read_csv("C.csv",header=None)
graph = df.as_matrix()

In [20]:
'''
Initiate Class for Shortest Path Problem
                                        
Input: 
    Graph - in form of matrix where Cuv is cost from edge u to edge v    
                                        
Methods:                                
    Dijkstras(start,end)- outputs shortest path and its cost     

    BellmanFord(start,end,max_steps) - output shortest path & cost for n steps 

    KshortestPaths(start,end,k) - outputs paths & cost for k shortest paths 
'''
class SPP():
    def __init__(self,graph):
        self.graph = graph

    #Algorithm for 1.A
    def Dijkstras(self,start,end):
        #initialize data structures:
        start -= 1
        end -= 1
        path_dict = dict()
        unexplored = set()
        
        for node in range(len(self.graph)):
            unexplored.add(node)
            path_dict[node] = [float('inf'),-1] #[cost , previous node in path]
        
        #Begin with starting node
        last_added = start
        path_dict[start] = [0,-1]
        #add to explored (take away from unexplored)
        unexplored.remove(start)
        
        while len(unexplored) != 0:
            
            #The Algorithm:
            #update costs by checking if newly "explored" nodes help out.
            #simultaneously we will keep track of smallest cost in unexplored
            #nodes, and add the minimum to list of explored nodes
            
            min_cost = [float("inf"),-1] #[cost,index]
            for v in unexplored:
                new_cost = path_dict[last_added][0] + self.graph[last_added][v]
                #update cost if less than old one
                if new_cost < path_dict[v][0]:
                    path_dict[v][0] = new_cost
                    path_dict[v][1] = last_added
                    
                if path_dict[v][0] < min_cost[0]:
                    min_cost[0] = path_dict[v][0]
                    min_cost[1] = v
            #add min cost to explored
            unexplored.remove(min_cost[1])
            last_added = min_cost[1]
            
        #Algorithm completed, now return shortest path
        sp = [end+1]
        condition = True
        try:
            previous = path_dict[end][1]
        except: 
            print("you're already there")
            condition = False
            
        i=0 #just in case
        while condition or i>10000:
            if previous == -1:
                break
            sp.append(previous+1)
            previous = path_dict[previous][1]
            i+=1
            
        return("Cost: " + str(path_dict[end][0]) + " Shortest Path: " + str(list(reversed(sp))))


    #Algorithm for 1.B ** big change here is no "Explored" set. Instead we check every possible node
    #Also, optimal costs and previous nodes are saved in each "step"
    def BellmanFord(self,start,end,max_steps = None):
        if max_steps == None:
            max_steps = len(self.graph)
        
        #initialize data structures:
        start -= 1
        end -= 1
        Length = max_steps+1
        path_dict = dict()
        
        for node in range(len(self.graph)):
            path_dict[node] = [[float('inf')]*Length,[-1]*Length] #[[cost indexed by step] , [previous node in path indexed by step]]
        
        #Begin with starting node
        last_added = start
        path_dict[start] = [[0]*Length,[-1]*Length]
        
        step = 0
        while step < max_steps:
            #The Algorithm:
            #update costs by checking every possible edge (much slower than Dijkstras)
            #update cost if d(u,k-1)+Cuv < d(v,k-1)
            for v in range(len(self.graph)):
                for u in range(len(self.graph)):
                    if u == v:
                        continue #dont check yourself please!  
                    new_cost = path_dict[u][0][step] + self.graph[u][v]
                    #update cost if less than old one AND current update
                    if (new_cost < path_dict[v][0][step]) and (new_cost < path_dict[v][0][step+1]):
                        path_dict[v][0][step+1] = new_cost
                        path_dict[v][1][step+1] = u
                if path_dict[v][0][step+1] > path_dict[v][0][step]:#No cost was less so keep old path
                    path_dict[v][0][step+1] = path_dict[v][0][step]
                    path_dict[v][1][step+1] = path_dict[v][1][step]
            step+=1
            
        #Algorithm completed, now return shortest path
        sp = [end+1] #paths are 1-indexed!
        condition = True
        try:
            previous = path_dict[end][1][max_steps]
        except: 
            print("you're already there")
            condition = False
            
        i=max_steps #Last "previous node" added 
        while condition or i>=0:
            if previous == -1:
                break
            sp.append(previous+1) #paths are 1-indexed!
            i-=1
            previous = path_dict[previous][1][i]
            
            
        return("Cost in " +str(max_steps) +" steps = " + str(path_dict[end][0][max_steps]) + "; Shortest Path: " + str(list(reversed(sp))))




In [19]:
#initialize shortest path problem with Graph
spp = SPP(graph)

#*******  ANSWER to 1.a  *********
spp.Dijkstras(1,20) #Cost: 24 Shortest Path: [1, 12, 4, 6, 8, 20]
spp.Dijkstras(11,13) #Cost: 12 Shortest Path: [11, 4, 16, 13]
spp.Dijkstras(17,3) #Cost: 24 Shortest Path: [17, 6, 8, 3]

#******* ANSWER to 1.B  **********
#1 -> 20 is the most interesting!
spp.BellmanFord(1,20,3) #'Cost in 3 steps = 100; Shortest Path: [1, 20]'
spp.BellmanFord(1,20,4) #'Cost in 4 steps = 27; Shortest Path: [1, 4, 6, 8, 20]'
spp.BellmanFord(1,20,5) #'Cost in 5 steps = 24; Shortest Path: [1, 12, 4, 6, 8, 20]'
spp.BellmanFord(11,13,3) #'Cost in 3 steps = 12; Shortest Path: [11, 4, 16, 13]'
spp.BellmanFord(11,13,5) #'Cost in 5 steps = 12; Shortest Path: [11, 4, 16, 13]'
spp.BellmanFord(17,3,3) #'Cost in 3 steps = 24; Shortest Path: [17, 6, 8, 3]'
spp.BellmanFord(17,3,5) #'Cost in 5 steps = 24; Shortest Path: [17, 6, 8, 3]'

'Cost in 5 steps = 24; Shortest Path: [17, 6, 8, 3]'

### PROBLEM 2.
Consider the Time-Dependent Shortest Path Problem. In the lecture notes, we assume
that for all τ1 < τ2, then τ1 + cuv(τ1) < τ2 + cuv(τ2).<br><br>
**a) Explain what this assumption means in simple language, and why it makes sense to assume it.**<br>
This means that we cannot arrive at a destination sooner by simply waiting until a later time to leave.  This is a natural and sound assumption because although the travel time may be shorter later, we can never do better than the person who is already on the road (but it may be preferable to watch tv and let traffic die down!)<br><br>
**b) Explain where Dijkstra’s Algorithm could fail if we do not have this assumption.**<br>
If this assumption did not apply, then Dijkstra's algorithm would fail by greedily adding a Node to the "Explored" set too early.  There is a similar consequence for the "no negative costs" assumption.<br><br>
### PROBLEM 3.
Consider the standard Shortest Path Problem.**<br>

**a) After we have solved the shortest path problem, we would now like to find the second shortest path (which may have the same cost). Provide a method to do this.**
Unfortunately this is a hard problem.  One slow way to do this would be to list all possible paths with an optimized algorithm.  Then start calculating costs for each path, and sort them.

A faster method would be to first compute the shortest path using Dijkstra's algorithm.  With this saved path (P) of E edges, we will iterate through each edge, making the cost infinity one by one.  We compute the shortest path E times and select the lowest.  This method allows for a path to use all but one of the original shortest path's edges (with a short detour), but will not allow a path to contain all of P.

**b) Your relative insists that a certain edge must be used on the shortest path. Provide a method that finds the shortest path that includes that edge.**
For this particular problem we can put a very negative cost on the required edge to incentivize using it.  However, now, we cannot use Dijkstra's Algorithm because a key assumption for it is non-negative costs (due to the greedy way in which it adds nodes to the "explored set"). Instead, we can use Bellman Ford's algorithm which has a longer runtime, but iteratively updates costs if a path is possible and shorter (i.e. if this negative path is found near the end of the graph). One key assumption we would need here is an **Acyclic** graph. 
